## Código baseado no [Github](https://github.com/felipe-matsuoka123/bert-enem-regression/blob/main/streamlit_bert_enem.py)

In [ ]:
import random
import numpy as np
import re
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel

In [ ]:
seed_val = 24

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
torch.backends.cudnn.deterministic = True

In [ ]:
# Montando o drive para a leitura de diretórios.
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Read corpus
# Load both train and test splits

train = pd.read_csv('corpus/train.csv')
test = pd.read_csv('corpus/test.csv')

In [ ]:
print(train.head(5))

    id                                              essay  \
0  753  [T] Os vidros de tintas\nOs vridos de tintas p...   
1  582  [T] O ARMÁRIO E TINTA MAGICA\nEU ENCONTREI CON...   
2  548  [T] Li um livro que me assustou !\n[P] Em uma ...   
3  113  Um dia eu vi mas latas de tinta em de [?]\nai ...   
4  174  [P]era uma vez uma menina Que esta [?] uque fa...   

                                              prompt  formal_register  \
0  Eu encontrei em cima do armário alguns potes c...                3   
1  Eu encontrei em cima do armário alguns potes c...                2   
2  Eu encontrei em cima do armário alguns potes c...                3   
3  Eu encontrei em cima do armário alguns potes c...                3   
4  Eu encontrei em cima do armário alguns potes c...                2   

   thematic_coherence  narrative_rhetorical_structure  cohesion  
0                   1                               1         1  
1                   2                               3         

In [ ]:
train.shape

(740, 7)

In [ ]:
# Remove tags

def get_tags(essays: pd.Series) -> np.ndarray:
    """Utility function to list all tags in
    a series of essays.
    """
    flags = []
    for e in essays:
        square = re.findall(r'(\[.{1,3}\])', e)
        curly = re.findall(r'(\{.{1,3}\})', e)
        mixed_1 = re.findall(r'(\[.{1,3}\})', e)
        mixed_2 = re.findall(r'(\{.{1,3}\])', e)

        for group in [square, curly, mixed_1, mixed_2]:
            flags.extend([g for g in group])

    return np.sort(np.unique(flags))

In [ ]:
# === Constructing the REGEX which removes all tags ===
# Well-formed tags with format [<LETTER_OR_SYMBOL>]
tag_regex =  r'(\[[PpSsTtXx?]\])'

# Well-formed tags with format {<LETTER_OR_SYMBOL>}
tag_regex += r'|({[ptx?]})'

# Well-formed tags [LT] or [LC]
tag_regex += r'|(\[L[TC]\])'

# Well-formed tags with format [lt] or [lc]
tag_regex += r'|(\[l[tc]\])'

# Variant with a trailing space
tag_regex += r'|(\[ P\])'

# Mixed closing/opening symbol
tag_regex += r'|(\[[PX?]\})'
tag_regex += r'|(\{?\])'

In [ ]:
# Replacing each occurrence of those tags with a blank
train.essay = train.essay.str.replace(tag_regex, '', regex=True)
test.essay = test.essay.str.replace(tag_regex, '', regex=True)

In [ ]:
print(train.head(5))

    id                                              essay  \
0  753   Os vidros de tintas\nOs vridos de tintas pint...   
1  582   O ARMÁRIO E TINTA MAGICA\nEU ENCONTREI CON TI...   
2  548   Li um livro que me assustou !\n Em uma (belea...   
3  113  Um dia eu vi mas latas de tinta em de \nai res...   
4  174  era uma vez uma menina Que esta  uque faze\ne ...   

                                              prompt  formal_register  \
0  Eu encontrei em cima do armário alguns potes c...                3   
1  Eu encontrei em cima do armário alguns potes c...                2   
2  Eu encontrei em cima do armário alguns potes c...                3   
3  Eu encontrei em cima do armário alguns potes c...                3   
4  Eu encontrei em cima do armário alguns potes c...                2   

   thematic_coherence  narrative_rhetorical_structure  cohesion  
0                   1                               1         1  
1                   2                               3         

In [ ]:
val = train.sample(frac=0.1,random_state=24)
new_train = train.drop(val.index)

In [ ]:
new_train.head(5)

,id,essay,prompt,formal_register,thematic_coherence,narrative_rhetorical_structure,cohesion
0,753,Os vidros de tintas\nOs vridos de tintas pint...,Eu encontrei em cima do armário alguns potes c...,3,1,1,1
1,582,O ARMÁRIO E TINTA MAGICA\nEU ENCONTREI CON TI...,Eu encontrei em cima do armário alguns potes c...,2,2,3,2
2,548,Li um livro que me assustou !\n Em uma (belea...,Eu encontrei em cima do armário alguns potes c...,3,1,5,3
3,113,Um dia eu vi mas latas de tinta em de \nai res...,Eu encontrei em cima do armário alguns potes c...,3,3,5,3
4,174,era uma vez uma menina Que esta uque faze\ne ...,Eu encontrei em cima do armário alguns potes c...,2,2,4,2


In [ ]:
val.head(5)

,id,essay,prompt,formal_register,thematic_coherence,narrative_rhetorical_structure,cohesion
61,943,o livro\nera uma vez eu e meu amigo estava jo...,Eu encontrei em cima do armário alguns potes c...,3,1,4,2
131,953,o quarto\n eu fui no meu amigo quando fui no ...,Eu encontrei em cima do armário alguns potes c...,2,1,2,2
220,425,As Lindas cores\n Numa tarde uma menina viu a...,Eu encontrei em cima do armário alguns potes c...,3,2,4,3
355,682,O montrinho do meu quarto\nEu encontrei em ci...,Eu encontrei em cima do armário alguns potes c...,3,2,4,3
505,727,OS ANIMAIS\n Uma Garota achou varias tintas n...,Eu encontrei em cima do armário alguns potes c...,3,3,4,3


In [ ]:
print(new_train.shape)
print(val.shape)

(666, 7)
(74, 7)


In [ ]:
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

In [ ]:
#Classe de manipulação de dataset:
class Dataset(Dataset):

    #Construtor da classe:
    def __init__(self, essays, prompts, tokenizer, labels, max_length=512):
        #Armazene as entradas que serão passadas ao modelo:
        self.essays = essays['essay'].values  # reviews["review_text"].values

        self.prompts = prompts

        #Armazene as labels que serão utilizadas para treino/validação/teste:
        self.labels = labels

        #Armazene o tokenizador:
        self.tokenizer = tokenizer

        #Armazene o tamanho máximo das sentenças:
        self.max_length = max_length

    #Retorna o número de instâncias:
    def __len__(self):
        return len(self.essays)

    #Retorna uma instância completa com base num índice:
    def __getitem__(self, index):
        #Obtenha a entrada do índice pertinente:
        #review = self.reviews[index]
        essay = self.essays[index]

        prompt = self.prompts[index]


        #Obtenha a label do índice pertinente:
        label = self.labels[index]

        #Tokenize a entrada:
        encoding = self.tokenizer(
          text_pair=prompt.tolist(),
          text=essay,
          add_special_tokens=True,
          max_length=self.max_length,
          return_token_type_ids=False,
          return_overflowing_tokens=False,
          padding='max_length',
          #pad_to_max_length=True,
          return_attention_mask=True,
          return_tensors='pt',
          truncation=True
        )

        #Obtenha os códigos numéricos da sentença:
        input_ids = encoding['input_ids'].flatten()

        #Obtenha os códigos numéricos dos token types:
        # token_type_ids = encoding['token_type_ids'].flatten()

        #Obtenha a máscara de atenção da sentença:
        attention_mask = encoding['attention_mask'].flatten()

        #Transforme a label da instância em um tensor:
        label_tensor = torch.tensor(label, dtype=torch.float)


        #Retorne um dicionário com estes dados:
        return {
          'input_ids': input_ids,
          #'token_type_ids': token_type_ids,
          'attention_mask': attention_mask,
          'labels': label_tensor

        }

In [ ]:
class BERTTimbauRegression(nn.Module):
    def __init__(self):
        super(BERTTimbauRegression, self).__init__()
        # Load the pre-trained BERT model
        self.bert = BertModel.from_pretrained('neuralmind/bert-large-portuguese-cased')
        # self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')

        # Dropout layer
        self.dropout = nn.Dropout(0.4)

        # A Linear layer to get 4 continuous values (PROPOR)

        self.regressor = nn.Linear(1024, 4)

    def forward(self, input_ids, attention_mask):
        # Get the output from BERT model
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)

        # Extract the last hidden state of the token `[CLS]` for classification task
        pooled_output = outputs.pooler_output

        # Apply dropout
        pooled_output = self.dropout(pooled_output)


        # Pass through the regressor
        return self.regressor(pooled_output)

In [ ]:
def round_to_nearest_grade(value):
    grades = [1, 2, 3, 4, 5]
    closest_grade = []
    for i in value:
        closest_grade.append(min(grades, key=lambda x: abs(x - i)))
    return closest_grade

In [ ]:
print(new_train.shape)
print(test.shape)
print(val.shape)

(666, 7)
(125, 7)
(74, 7)


In [ ]:
max_length = 512
batch_size = 8  # 16

y_train = new_train[['formal_register', 'thematic_coherence', 'narrative_rhetorical_structure', 'cohesion']].values
y_test = test[['formal_register', 'thematic_coherence', 'narrative_rhetorical_structure', 'cohesion']].values
y_val = val[['formal_register', 'thematic_coherence', 'narrative_rhetorical_structure', 'cohesion']].values

train_prompts = new_train[['prompt']].values
val_prompts = val[['prompt']].values
test_prompts = test[['prompt']].values

train_set = Dataset(new_train, train_prompts, tokenizer, y_train, max_length)
train_loader = DataLoader(train_set, batch_size = batch_size, shuffle=True)

test_set = Dataset(test, test_prompts, tokenizer, y_test, max_length)
test_loader = DataLoader(test_set, batch_size = batch_size, shuffle=False)

val_set = Dataset(val, val_prompts, tokenizer, y_val, max_length)
val_loader = DataLoader(val_set, batch_size = batch_size, shuffle=False)

In [ ]:
val[['prompt']].values[0]

array(['Eu encontrei em cima do armário alguns potes com tinta. Então resolvi pintar na parede do meu quarto alguns animais. Tempo depois, quando voltei no quarto, não havia mais nenhuma pintura e as roupas da gaveta estavam espalhadas pelo chão.'],
      dtype=object)


DEBUG

In [ ]:
#Crie um dataset de testes:
dtoy = Dataset(val, train_prompts, tokenizer, y_train, max_length=45)

#Pegue uma instância do dataset:
data_inst = next(iter(dtoy))

#Imprima os componentes da instância:
print("Input IDs:", data_inst['input_ids'])
# print("Token Type IDs:", data_inst['token_type_ids'])
print("Attention Mask:", data_inst['attention_mask'])
print("Labels:", data_inst['labels'])

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Input IDs: tensor([  101,   146,  1722,   495,   230,   576,  2779,   122,  7343,  3695,
         1011,  9572,  3733,   122,  2080,   230,  2760,  3762, 22324,  2472,
          368,  1833,   456,   959,  6281,   170,   202,   145,   122,  2779,
          122,  7343,  3695,  1011, 18224,   243,   123, 22283,   368,  5608,
         2779,   122,   102,   100,   102])
Attention Mask: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
Labels: tensor([3., 1., 1., 1.])


In [ ]:
print(y_val[0])

[3 1 4 2]


In [ ]:
# Crie o modelo:
from transformers import AdamW, get_linear_schedule_with_warmup

device = "cuda" if torch.cuda.is_available() else "cpu"
BertPROPOR = BERTTimbauRegression()
BertPROPOR.to(device)

epochs = 6
total_steps = len(train_loader)*epochs
loss_function = torch.nn.MSELoss()
optimizer = AdamW(BertPROPOR.parameters(), lr=4e-5, eps=1e-8)

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import tqdm
def train_model(model, data_loader, loss_fn, optimizer, scheduler):
    #Coloque o modelo em modo de treinamento:
    model.train()

    #Inicialize o erro total da epoch:
    total_loss = 0
    total_preds = []

    #Para cada batch do data_loader, faça:
    for d in tqdm.tqdm(data_loader, desc='training...'):
        #Obtenha os dados da batch:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels = d["labels"].to(device)

        #Passe os dados pelo modelo:
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        # for o in outputs:
        #     print(o.item())
        # outputs = outputs.cpu().detach().numpy()
        # print(outputs)
        # exit()
        # Obtenha as predições:
        # _, preds = torch.max(outputs, dim=1)
        # total_preds.extend([p.item() for p in preds])
        # print(total_preds)
        # print(outputs)
        # outputs = [round_to_nearest_grade(score) for score in outputs]
        # print(outputs)
        # outputs = np.array(outputs)
        # outputs = torch.FloatTensor(outputs).cuda()
        # outputs = torch.from_numpy(np.array(outputs)).float().to(device)
        #Calcule o erro:
        loss = loss_fn(outputs, labels)
        total_loss += loss.item()

        #Propague o erro para o modelo, promovendo aprendizado:
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return total_loss / len(data_loader)

In [ ]:
def test_model(model, data_loader, loss_fn):
    #Coloque o modelo em modo de treinamento:
    model.eval()

    #Inicialize o erro total da epoch:
    total_loss = 0
    total_preds = []
    with torch.no_grad():
      #Para cada batch do data_loader, faça:
      for d in tqdm.tqdm(data_loader, desc='evaluating...'):
          #Obtenha os dados da batch:
          input_ids = d["input_ids"].to(device)
          attention_mask = d["attention_mask"].to(device)
          labels = d["labels"].to(device)

          #Passe os dados pelo modelo:
          outputs = model(input_ids=input_ids, attention_mask=attention_mask)

          #Obtenha as predições:
          #_, preds = torch.max(outputs, dim=1)
          # total_preds.extend([p.item() for p in preds])

          #Calcule o erro:
          loss = loss_fn(outputs, labels)
          total_loss += loss.item()

          # preds = outputs # .squeeze(1)
          preds = outputs.cpu().detach().numpy()
          total_preds.extend(preds)

    return total_loss / len(data_loader), total_preds

In [ ]:
import os
output_dir = 'model_save/'


# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


In [ ]:
#Treine o modelo:

best_valid_loss = float('inf')

for i in range(epochs):
    #Treine em dados de treinamento:
    #print('\nTreinando o modelo, epoch ', i)
    total_loss_tr = train_model(BertPROPOR, train_loader, loss_function, optimizer, scheduler)

    #Valide em dados de validação:
    #print('Validando o modelo, epoch ', i)
    total_loss_va, _ = test_model(BertPROPOR, val_loader, loss_function)

    if total_loss_va < best_valid_loss:
      best_valid_loss = total_loss_va
      torch.save(BertPROPOR.state_dict(), output_dir + 'model_finetuned_bert.pt')

    #Imprima os erros de treinamento/validação:
    print('Erro de treinamento:', total_loss_tr)
    print('Erro de validação:', total_loss_va)

evaluating...: 100%|██████████| 10/10 [00:01<00:00,  5.03it/s]


Erro de treinamento: 1.2320349927814234
Erro de validação: 0.48657396137714387


training...:  45%|████▌     | 38/84 [00:22<00:27,  1.67it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
evaluating...: 100%|██████████| 10/10 [00:01<00:00,  5.07it/s]


Erro de treinamento: 0.5276726118865467
Erro de validação: 0.3422055870294571


evaluating...: 100%|██████████| 10/10 [00:01<00:00,  5.05it/s]


...

Erro de treinamento: 0.3497583186697392
Erro de validação: 0.31238372921943663


evaluating...: 100%|██████████| 10/10 [00:01<00:00,  5.06it/s]


Erro de treinamento: 0.296263369598559
Erro de validação: 0.285944801568985


evaluating...: 100%|██████████| 10/10 [00:01<00:00,  5.05it/s]

Erro de treinamento: 0.2713231611996889
Erro de validação: 0.28969732075929644


In [ ]:
BertPROPOR.load_state_dict(torch.load(output_dir + 'model_finetuned_bert.pt'))

<All keys matched successfully>

In [ ]:
from sklearn.metrics import f1_score, cohen_kappa_score

#Teste o modelo:
total_loss_te, preds_te = test_model(BertPROPOR, test_loader, loss_function)
# print(preds_te)
# preds_te = torch.stack(preds_te).cpu().numpy()
outputs = [round_to_nearest_grade(score) for score in preds_te]
# print(preds_te)
# print(y_test[:,0])
c1 = [item[0] for item in outputs]
c2 = [item[1] for item in outputs]
c3 = [item[2] for item in outputs]
c4 = [item[3] for item in outputs]
print(c1)

fscores = []
fscores.append(f1_score(y_test[:,0], c1, average="weighted"))
fscores.append(f1_score(y_test[:,1], c2, average="weighted"))
fscores.append(f1_score(y_test[:,2], c3, average="weighted"))
fscores.append(f1_score(y_test[:,3], c4, average="weighted"))

kappa = []
kappa.append(cohen_kappa_score(y_test[:,0], c1))
kappa.append(cohen_kappa_score(y_test[:,1], c2))
kappa.append(cohen_kappa_score(y_test[:,2], c3))
kappa.append(cohen_kappa_score(y_test[:,3], c4))


print(f'\n Média: {np.mean([score for score in fscores] + [value for value in kappa])}')

# fc1 = f1_score(y_test[:,0], c1, average='weighted')

print(f'\nF-score C1: {f1_score(y_test[:,0], c1, average="weighted")}')
print(f'F-score C2: {f1_score(y_test[:,1], c2, average="weighted")}')
print(f'F-score C3: {f1_score(y_test[:,2], c3, average="weighted")}')
print(f'F-score C4: {f1_score(y_test[:,3], c4, average="weighted")}')

print('\n')

print(f'Kappa C1: {cohen_kappa_score(y_test[:,0], c1)}')
print(f'Kappa C2: {cohen_kappa_score(y_test[:,1], c2)}')
print(f'Kappa C3: {cohen_kappa_score(y_test[:,2], c3)}')
print(f'Kappa C4: {cohen_kappa_score(y_test[:,3], c4)}')
# print(y_test[:,0])

# print('Labels reais: ', y_test)
# print('Labels preditas: ', preds_te)

#Calcule o erro com as etiquetas reais:
# print(classification_report(y_test, outputs))

evaluating...: 100%|██████████| 16/16 [00:03<00:00,  4.85it/s]

[3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 2, 4, 3, 3, 3, 3, 4, 3, 2, 3, 3, 3, 3, 2, 3, 3, 4, 3, 2, 4, 3, 4, 3, 2, 2, 3, 3, 3, 4, 3, 4, 3, 3, 3, 2, 3, 4, 3, 3, 3, 3, 4, 3, 3, 3, 3, 4, 3, 3, 3, 4, 2, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 2, 4, 3, 3, 3, 3, 4, 4, 3, 4, 3, 3, 3, 3, 2, 3, 4, 4, 4, 2, 2, 3, 3, 2, 3, 4, 3, 2, 4, 2, 3, 2, 3, 3, 4, 4, 4, 4, 4, 3, 2, 3, 3, 4]

 Média: 0.5082942603886162

F-score C1: 0.685824509883895
F-score C2: 0.6590781224004751
F-score C3: 0.544211834776876
F-score C4: 0.6610680359435175


Kappa C1: 0.45074005550416274
Kappa C2: 0.5308688309251266
Kappa C3: 0.1919746310525674
Kappa C4: 0.34258806262230923


In [ ]:
ids = test.id.tolist()
data = {
    'id': ids,
    'formal_register': c1,
    'thematic_coherence':c2,
    'narrative_rhetorical_structure': c3,
    'cohesion': c4}

df = pd.DataFrame(data)
df.to_csv('submission.csv', index=False)